<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/322_EaaS_CostROI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_evaluation_cost(
    evaluation: Dict[str, Any],
    cost_per_second: float = 0.001,  # $0.001 per second of execution
    api_cost_per_evaluation: float = 0.01  # $0.01 per API call
) -> float:
    """Calculate cost for a single evaluation."""
    execution_time = evaluation.get('execution_time_seconds', 0.0)
    time_cost = execution_time * cost_per_second
    api_cost = api_cost_per_evaluation
    return time_cost + api_cost


def estimate_revenue_impact(
    agent_id: str,
    evaluations: List[Dict[str, Any]],
    agent_value_map: Optional[Dict[str, float]] = None
) -> float:
    """
    Estimate revenue impact for an agent based on successful evaluations.

    MVP: Simple estimation based on agent type and success rate.
    Future: More sophisticated calculation based on actual business metrics.
    """
    if agent_value_map is None:
        # Default value per successful evaluation by agent type
        agent_value_map = {
            'shipping_update_agent': 25.0,  # Prevents churn, improves satisfaction
            'refund_agent': 50.0,  # Customer retention value
            'apology_message_agent': 15.0,  # Customer satisfaction value
            'escalation_agent': 75.0,  # Issue resolution value
        }

    agent_evaluations = [e for e in evaluations if e.get('target_agent_id') == agent_id]
    successful = [e for e in agent_evaluations if e.get('status') == 'completed']

    value_per_action = agent_value_map.get(agent_id, 20.0)  # Default $20
    success_rate = len(successful) / len(agent_evaluations) if agent_evaluations else 0.0

    # Revenue impact = successful actions × value per action
    revenue_impact = len(successful) * value_per_action * success_rate

    return revenue_impact


In [ ]:
def calculate_agent_performance_summary(
    agent_id: str,
    evaluations: List[Dict[str, Any]],
    scores: List[Dict[str, Any]],
    health_thresholds: Dict[str, float],
    include_roi: bool = True
) -> Dict[str, Any]:
    """Calculate performance summary for an agent, including ROI if requested."""
    agent_evaluations = [e for e in evaluations if e.get('target_agent_id') == agent_id]
    agent_scores = [s for s in scores if s.get('target_agent_id') == agent_id]

    if not agent_scores:
        return {
            "agent_id": agent_id,
            "total_evaluations": 0,
            "passed_count": 0,
            "failed_count": 0,
            "average_score": 0.0,
            "average_response_time": 0.0,
            "health_status": "unknown"
        }

    total = len(agent_scores)
    passed = sum(1 for s in agent_scores if s.get('passed', False))
    failed = total - passed
    avg_score = sum(s.get('overall_score', 0.0) for s in agent_scores) / total

    avg_response_time = sum(
        e.get('execution_time_seconds', 0.0) for e in agent_evaluations
    ) / len(agent_evaluations) if agent_evaluations else 0.0

    # Determine health status
    if avg_score >= health_thresholds.get('healthy', 0.85):
        health_status = "healthy"
    elif avg_score >= health_thresholds.get('degraded', 0.70):
        health_status = "degraded"
    else:
        health_status = "critical"

    result = {
        "agent_id": agent_id,
        "total_evaluations": total,
        "passed_count": passed,
        "failed_count": failed,
        "average_score": avg_score,
        "average_response_time": avg_response_time,
        "health_status": health_status
    }

    # Add ROI calculations if requested
    if include_roi:
        # Calculate costs
        total_cost = sum(calculate_evaluation_cost(e) for e in agent_evaluations)

        # Estimate revenue impact
        revenue_impact = estimate_revenue_impact(agent_id, agent_evaluations)

        # Calculate ROI
        net_roi = revenue_impact - total_cost
        roi_ratio = revenue_impact / total_cost if total_cost > 0 else float('inf')
        roi_percent = ((revenue_impact - total_cost) / total_cost * 100) if total_cost > 0 else float('inf')

        # Determine ROI category
        if roi_percent >= 200:
            roi_category = "exceptional"
        elif roi_percent >= 100:
            roi_category = "high"
        elif roi_percent >= 50:
            roi_category = "medium"
        elif roi_percent >= 0:
            roi_category = "low"
        else:
            roi_category = "negative"

        result.update({
            "total_cost": round(total_cost, 2),
            "revenue_impact": round(revenue_impact, 2),
            "net_roi": round(net_roi, 2),
            "roi_ratio": round(roi_ratio, 2) if roi_ratio != float('inf') else float('inf'),
            "roi_percent": round(roi_percent, 2) if roi_percent != float('inf') else float('inf'),
            "roi_category": roi_category,
            "cost_per_evaluation": round(total_cost / total, 2) if total > 0 else 0.0
        })

    return result